In [20]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks

# Load Data
train_df = pd.read_csv("modified_data_train.csv")
test_df = pd.read_csv("modified_data_test.csv")

print("Train shape:", train_df.shape)
print("Test shape:", test_df.shape)

# Split features and target
TARGET_COL = 'faultNumber'  
X_train = train_df.drop(columns=[TARGET_COL])
y_train = train_df[TARGET_COL]

X_test = test_df.drop(columns=[TARGET_COL])
y_test = test_df[TARGET_COL]


Train shape: (25920, 41)
Test shape: (50760, 41)


In [21]:
# Encode target if categorical
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

In [22]:
# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [23]:
# Build ANN model
def build_ann(input_dim, num_classes):
    model = models.Sequential()
    model.add(layers.Input(shape=(input_dim,)))
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dropout(0.2))
    
    # Output layer based on classification type
    if num_classes == 2:
        model.add(layers.Dense(1, activation='sigmoid'))  # Binary
        loss = 'binary_crossentropy'
        metrics = ['accuracy']
    else:
        model.add(layers.Dense(num_classes, activation='softmax'))  # Multi-class
        loss = 'sparse_categorical_crossentropy'
        metrics = ['accuracy']
    
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss=loss,
        metrics=metrics
    )
    return model

model = build_ann(X_train_scaled.shape[1], num_classes)
model.summary()

# Callbacks
early_stop = callbacks.EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True)
reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=4, min_lr=1e-6)

# Train Model
history = model.fit(
    X_train_scaled, y_train,
    validation_split=0.2,
    epochs=50,
    batch_size=32,
    callbacks=[early_stop, reduce_lr],
    verbose=1
)

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_12 (Dense)                │ (None, 128)            │         5,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 18)             │         1,170 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,674 (57.32 KB)

 Trainable params: 14,674 (57.32 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
648/648 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.3351 - loss: 2.1649 - val_accuracy: 0.2895 - val_loss: 2.2859 - learning_rate: 0.0010
Epoch 2/50
648/648 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.5767 - loss: 1.2511 - val_accuracy: 0.3690 - val_loss: 2.2193 - learning_rate: 0.0010
Epoch 3/50
648/648 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.6597 - loss: 0.9944 - val_accuracy: 0.3831 - val_loss: 2.5680 - learning_rate: 0.0010
Epoch 4/50
648/648 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.7031 - loss: 0.8346 - val_accuracy: 0.3706 - val_loss: 3.1976 - learning_rate: 0.0010
Epoch 5/50
648/648 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.7227 - loss: 0.7704 - val_accuracy: 0.4012 - val_loss: 3.1238 - learning_rate: 0.0010
Epoch 6/50
648/648 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.7435 - loss: 0.6983 - val_accuracy: 0.3962 - val_loss: 3.4094 - learning_rate: 0.0010
Epoch 7/50
648/648 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.7597 - loss: 0.6591 - 

In [24]:
# Evaluate Model
if num_classes == 2:
    y_pred_prob = model.predict(X_test_scaled)
    y_pred = (y_pred_prob > 0.5).astype(int).flatten()
else:
    y_pred_prob = model.predict(X_test_scaled)
    y_pred = np.argmax(y_pred_prob, axis=1)

# Metrics
print("\n--- Evaluation Results ---")
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred, target_names=le.classes_.astype(str)))

# Save Model and Scaler
model.save("fault_detection_ann_model.keras")

import joblib
joblib.dump(scaler, "fault_scaler.joblib")
joblib.dump(le, "fault_label_encoder.joblib")

print("\n Model, Scaler, and Label Encoder saved successfully!")

1587/1587 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step

--- Evaluation Results ---
Accuracy: 0.5063829787234042

Confusion Matrix:
 [[1605    0    0   58  198    0    0    0  194  162    0    0    0  335
     0    0  220   48]
 [ 205 2351    0   34   41    0    0   18   38   28    0    0    1   34
     0    0   45   25]
 [ 217    0 2356   35   34    0    0    6   40   27    0    1    0   38
     0    0   48   18]
 [ 224    0    0 2348   44    0    0    0   36   28    0    0    3   39
     0    0   80   18]
 [1248    0    0   52  551    0    1   35  180  133   81   19    0  296
     0    0  179   45]
 [ 198    0    0  272   31 1615  528    0   36   25   12    4    0   34
     0    0   44   21]
 [ 198    0    0   34   31    0 2400    0   36   25    0    0    0   34
     0    0   44   18]
 [ 236   51   12   36  183    0    0 1567   50   30  471   13    0   50
     0    1   51   69]
 [1246    0    0   58  526    0    0    0  249  145    0    1    0  314
     0    0  198   83]
 [ 706    0    0  524  13